In [78]:
from Constants import REVIEWS_BASEDIR, SENTIMENTS, REVIEWS_IGNORE_TAGS
from Corpora import MovieReviewCorpus

import os
from nltk.stem.porter import PorterStemmer
from glob import glob

ImportError: cannot import name 'REVIEWS_IGNORE_TAGS' from 'Constants' (/Volumes/GoogleDrive/My Drive/Cambridge University/MLMI/Course/L90 - Natural Language Processing/MLMI13/Constants.py)

In [2]:
with open("data/reviews/NEG/cv000_29416.txt") as f:
    print(f.readlines()[:10])

['Two teen couples go to a church party, drink and then drive. \n', '\n', 'They get into an accident. \n', '\n', 'One of the guys dies, but his girlfriend continues to see him in her life, and has nightmares. \n', '\n', "What's the deal? \n", '\n', 'Watch the movie and "sorta" find out... \n', '\n']


In [3]:
with open("data/reviews/NEG/cv000_29416.tag") as f:
    lines = f.readlines()
    print(lines[:20])
    print(lines[0].strip())

['Two\tCD\n', 'teen\tJJ\n', 'couples\tNNS\n', 'go\tVBP\n', 'to\tTO\n', 'a\tDT\n', 'church\tNN\n', 'party\tNN\n', ',\t,\n', 'drink\tNN\n', 'and\tCC\n', 'then\tRB\n', 'drive\tNN\n', '.\t.\n', '\n', 'They\tPRP\n', 'get\tVBP\n', 'into\tIN\n', 'an\tDT\n', 'accident\tNN\n']
Two	CD


In [4]:
stemmer = PorterStemmer()

In [38]:
IGNORE_TAGS = {"\n"}

In [39]:
def process_tag(tag, stem: bool = False):
    stripped = tag.strip()

    if stripped:
        split = stripped.split()
        if len(split) == 2:
            word, pos_tag = stripped.split()
        else:
            return False, tag
    else:
        # Return the non-processed tag
        return False, tag
    
    if stem:
        token = stemmer.stem(word)
    else:
        token = word.lower()
    return True, (token, pos_tag)

In [58]:
def process_file(file, stem: bool = False):
    token_tags = []
    rejects = []
    with open(file) as f:
        entries = (process_tag(l, stem=stem) for l in f.readlines() if l not in IGNORE_TAGS)
        for e in entries:
            success, result = e
            if success:
                token_tags.append(result)
            else:
                rejects.append(result)
    return token_tags, rejects

In [41]:
token_tags, rejects = process_file("data/reviews/NEG/cv000_29416.tag")

In [65]:
os.path.basename("data/reviews/NEG/cv000_29416.tag")[:3]

'cv0'

In [42]:
token_tags[:10]

[('two', 'CD'),
 ('teen', 'JJ'),
 ('coupl', 'NNS'),
 ('go', 'VBP'),
 ('to', 'TO'),
 ('a', 'DT'),
 ('church', 'NN'),
 ('parti', 'NN'),
 (',', ','),
 ('drink', 'NN')]

In [43]:
rejects

[]

In [67]:
results = []
rejects = []
errored_fles = []

for folder in SENTIMENTS:
    files = glob(os.path.join(REVIEWS_BASEDIR, folder, "*.tag"))
    for file in files:
        try:
            token_tags, rejected_tags = process_file(file, stem=True)
            results.append((folder, token_tags))
            if rejected_tags:
                rejects.append((file, rejected_tags))
        except Exception as e:
            errored_fles.append((e, file))
            continue

In [60]:
errored_fles

[]

In [61]:
rejects

[('data/reviews/POS/cv162_10424.tag',
  ['<director\xa0and\xa0screenwriter>\tNN\n']),
 ('data/reviews/POS/cv265_10814.tag', ['2\xa01/2\tCD\n']),
 ('data/reviews/POS/cv717_15953.tag', ['2\xa01/2\tCD\n']),
 ('data/reviews/POS/cv819_9364.tag', ['3\xa01/2\tCD\n']),
 ('data/reviews/POS/cv843_15544.tag', ['289\xa0460\xa0939\tCD\n']),
 ('data/reviews/NEG/cv560_18608.tag', ['33\xa01/3\tCD\n'])]

In [62]:
len(results)

2000